In [25]:
import torch
from torchvision import transforms
from PIL import Image
import torchvision.models as models

In [26]:
img = Image.open('n01440764_tench.JPEG')

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
input_batch = input_batch.to(device)
input_batch

tensor([[[[-0.8335, -0.8849, -0.8678,  ..., -0.5424, -0.4226, -0.3541],
          [-0.7993, -0.7993, -0.7822,  ..., -0.5596, -0.4911, -0.3712],
          [-0.8335, -0.8335, -0.7993,  ..., -0.5424, -0.5424, -0.4226],
          ...,
          [-0.9534, -0.9705, -0.9705,  ..., -0.7650, -0.6794, -0.6452],
          [-0.9877, -0.9877, -0.9363,  ..., -0.7479, -0.6452, -0.6109],
          [-0.9192, -0.9363, -0.9705,  ..., -0.6965, -0.5938, -0.6281]],

         [[-0.4426, -0.5126, -0.4951,  ..., -0.1800, -0.0924, -0.0399],
          [-0.4076, -0.4426, -0.4076,  ..., -0.1625, -0.1099, -0.0399],
          [-0.4076, -0.4426, -0.3725,  ..., -0.2150, -0.1625, -0.0399],
          ...,
          [-0.6352, -0.6527, -0.6176,  ..., -0.4076, -0.3725, -0.3200],
          [-0.6001, -0.5826, -0.5826,  ..., -0.3725, -0.3375, -0.3025],
          [-0.6001, -0.6352, -0.6176,  ..., -0.3375, -0.2850, -0.2850]],

         [[-0.3055, -0.4101, -0.3404,  ..., -0.0615,  0.0082,  0.0605],
          [-0.2532, -0.3230, -

In [27]:
arch = 'squeezenet1_0'
model_names = models.list_models(module=models)
# Load the model onto the device
model = models.__dict__[arch](pretrained=True)
model.to(device)
model.eval()
# loaded_scripted_model.to(device)

# Perform inference with the original quantized model
with torch.no_grad():
    original_output = model(input_batch)

In [28]:
# 使用 TorchScript 加载模型
scripted_model_name = f"quantized_{arch}_v1.pt"
loaded_scripted_model = torch.jit.load(scripted_model_name)
loaded_scripted_model.eval()
loaded_scripted_model.state_dict()

# Perform inference with the loaded quantized model
with torch.no_grad():
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension
    loaded_output = loaded_scripted_model(input_batch)


In [29]:
# Find the maximum value and its index
original_max_value, original_max_index = torch.max(original_output, dim=1)
loaded_max_value, loaded_max_index = torch.max(loaded_output, dim=1)

# Compare the maximum values and their indices
print("Original Max Value:", original_max_value)
print("Original Max Index:", original_max_index)
print("Loaded Max Value:", loaded_max_value)
print("Loaded Max Index:", loaded_max_index)
print("Max Value Difference:", torch.abs(original_max_value.cpu() - loaded_max_value))
print("Max Index Difference:", original_max_index.cpu() != loaded_max_index)

Original Max Value: tensor([33.1787])
Original Max Index: tensor([0])
Loaded Max Value: tensor([33.5349])
Loaded Max Index: tensor([0])
Max Value Difference: tensor([0.3562])
Max Index Difference: tensor([False])
